In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from src.inference import get_feature_store

In [3]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-02 22:34:59,732 INFO: Initializing external client
2025-03-02 22:34:59,733 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 22:35:00,535 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Fetching data from 2025-02-02 03:34:59.731574+00:00 to 2025-03-03 02:34:59.731574+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.41s) 


In [4]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 04:00:00+00:00,2,0
1,2025-02-02 05:00:00+00:00,2,0
2,2025-02-02 06:00:00+00:00,2,0
3,2025-02-02 07:00:00+00:00,2,0
4,2025-02-02 08:00:00+00:00,2,0
...,...,...,...
174440,2025-03-02 22:00:00+00:00,263,45
174441,2025-03-02 23:00:00+00:00,263,17
174442,2025-03-03 00:00:00+00:00,263,12
174443,2025-03-03 01:00:00+00:00,263,13


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 177456
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [6]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 177456
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [8]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [9]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,1,6,8,0,13,2,0,1,1,...,4,5,0,7,7,0,1,7,33,2025-03-02 19:00:00
1,139,39,8,261,168,8,190,15,235,201,...,135,122,113,11,97,110,13,10,162,2025-03-02 01:00:00
2,99,38,65,30,15,58,9,18,105,12,...,125,127,176,307,64,2,165,0,114,2025-03-02 12:00:00
3,1,0,3,1,4,0,0,1,0,0,...,0,1,4,1,2,2,4,0,93,2025-03-02 14:00:00
4,112,127,41,12,35,150,88,126,57,91,...,140,154,137,82,114,69,29,147,263,2025-03-02 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,94,2025-03-02 16:00:00
247,7,10,7,22,15,3,7,7,32,25,...,19,17,15,14,3,4,18,12,41,2025-03-02 16:00:00
248,2,0,3,1,2,3,0,1,1,0,...,2,0,0,1,1,1,1,0,37,2025-03-02 01:00:00
249,1,0,0,0,1,0,0,0,0,2,...,1,0,0,1,3,0,0,0,62,2025-03-02 09:00:00


In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-02 22:35:26,955 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 22:35:26,958 INFO: Initializing external client
2025-03-02 22:35:26,958 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 22:35:27,660 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Fetching data from 2025-02-02 03:35:26.955686+00:00 to 2025-03-03 02:35:26.955686+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.30s) 


In [11]:
current_date

Timestamp('2025-03-03 03:35:26.955686+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-02 04:00:00
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-02 04:00:00
2,7,1,0,2,2,3,0,4,3,2,...,4,11,14,50,68,62,65,43,4,2025-03-02 04:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-02 04:00:00
4,1,1,2,1,1,6,4,3,1,3,...,3,2,1,2,3,1,1,3,7,2025-03-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-02 04:00:00
247,0,0,2,4,2,1,1,2,0,3,...,1,1,0,1,0,0,0,0,260,2025-03-02 04:00:00
248,2,1,3,1,9,14,9,30,28,30,...,23,20,11,14,7,12,7,10,261,2025-03-02 04:00:00
249,3,6,11,30,44,59,85,107,91,91,...,58,26,28,18,7,5,5,2,262,2025-03-02 04:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-02 22:35:46,149 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 22:35:46,151 INFO: Initializing external client
2025-03-02 22:35:46,151 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 22:35:46,870 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [15]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,19.0
3,6,0.0
4,7,5.0
...,...,...
246,259,0.0
247,260,1.0
248,261,4.0
249,262,4.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([ 79, 148, 249,  48, 144, 246, 114,  68, 230,  90], dtype=int32)